# 3. extract 1 file of h5 images -> useable data

this notebook is for extracting individual crystals from the large h5 files, and converting to their own individual png + have a corresponding csv with a bunch of stats about the particle

(notebook follows on from 2. which has more comments + details)

* one file at a time -> potentially loop in the future
* assume Jonny format, all images stacked together
* !! importantly, using Jaffeux et al. 2022 paper for processing details !!

In [20]:
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
from glob import glob
import seaborn as sns
import xesmf as xe
import pandas as pd
#import def_homebrew as hb ## homemade functions xox
from scipy.special import gamma
import netCDF4 as nc
from datetime import datetime, timedelta

import h5py ####
from PIL import Image
#from IPython.display import display #
#import cv2 # not working
import os

from scipy.ndimage import convolve, label
from skimage.measure import regionprops, find_contours
from scipy.spatial import ConvexHull, distance_matrix
from skimage.morphology import remove_small_holes ## remove holes <3
from scipy.ndimage import binary_fill_holes
from skimage import measure
from cv2 import cvtColor, COLOR_BGR2GRAY, threshold, THRESH_BINARY, THRESH_OTSU

In [12]:
## files location
ds_loc = '/home/users/esree/data/2ds/'
hvps_loc = '/home/users/esree/data/hvps/'

#file of interest
file_name = 'Export_base220730153000.h5' # example file
f2ds = h5py.File(ds_loc+ file_name,'r') # open file

# break file into two - data + time
ds_image = f2ds['ImageData'] 
ds_time = f2ds['ImageTimes']
## ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
##### make xarray of useful time data #####
sec_since = ds_time[:,0]
pixel_slice = ds_time[:,1]
pix_sum = pixel_slice.cumsum(dtype = 'int')

## make useful datetime format (not seconds since midnight)
# using the file name for reference
date_str = file_name[11:17]
starting_date = datetime.strptime(date_str, '%y%m%d')
time_deltas = [timedelta(seconds=float(sec)) for sec in sec_since]
utc_time = [starting_date + delta for delta in time_deltas]

time_xr =xr.Dataset({
    'utc_time':utc_time,
    'pixel_slice': pixel_slice,
    'pix_sum': pix_sum})
## ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

In [13]:
## this is the code for splitting up the big h5 file -> individual images
save_path = '/gws/nopw/j04/dcmex/users/ezriab/2dprocessed/'
folder_name = f'flight_{file_name[11:23]}' # each flightset -> own folder

if not os.path.exists(save_path+folder_name):
    os.makedirs(save_path+folder_name)
    print("Folder created successfully!")
else:
    print("Folder already exists.")
save_loc = save_path+folder_name+'/'

Folder already exists.


In [14]:
## functions to make this run smoothly
def stats_description(bw_crystal):
    '''take binary image, fill in small holes and returns object containing stats about crystal'''
    
    filled_particle = remove_small_holes(bw_crystal.image, area_threshold=4) # fill in voids within binary image - better estimation of stats # may need to be altered
    
    # can see the filled in particle if needs be
    #plt.imshow(filled_particle, cmap='gray')
    
    contours = measure.find_contours(filled_particle, 0.5)
    if contours:
        contour = max(contours, key=lambda x: x.shape[0])  # Sort contours by area (largest first) and select the largest contour
        
        labeled_image = measure.label(filled_particle)  # Label the image based on the threshold
        region = measure.regionprops(labeled_image)[0]  # Assumes largest labeled region corresponds to largest contour
        
        return region
    else:
        return None

## ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# function to add padding to extracted crystal - better sample to match Jaffeux
def add_padding(one_crystal, desired_png_size):
    # Calculate padding
    pad_height = (desired_png_size - one_crystal.shape[0]) // 2
    pad_width = (desired_png_size - one_crystal.shape[1]) // 2
    
    # Pad the image
    padded_image = np.pad(one_crystal, ((pad_height, pad_height), (pad_width, pad_width)), mode='constant', constant_values=255)
    
    # In case the dimensions are odd, add an extra row/column to match the desired size
    if padded_image.shape[0] < desired_size:
        padded_image = np.pad(padded_image, ((0, 1), (0, 0)), mode='constant', constant_values=255)
    if padded_image.shape[1] < desired_size:
        padded_image = np.pad(padded_image, ((0, 0), (0, 1)), mode='constant', constant_values=255)

    return padded_image

In [15]:
## cleaning of whole h5 file
diff = np.diff(time_xr['pix_sum'][:].values) # this is finding the difference between the elements of pix_sum
selected_values = time_xr['pix_sum'][:-1][diff > 4] # this is selecting the adjacent files in which have pixels > 4 length

## actual bits about extraction of images - possibly need to edit

In [16]:
length_threshold = 300 # mu - need this minimum length of max dimension to extract the particle
pixel_resolution = 10 # mu
desired_image_size = 120

In [22]:
minimum_area = 15 # very quick metric to stop the processing of particles with area < 10 pixels

###  set up dataframe, used to extract from raw h5 file + has stats about the particle
columns = [
    "name",
    "particle_label",
    "start_index",
    "end_index",
    "start_time",
    "end_time",
    "major_axis_length",
    "minor_axis_length",
    "orientation",
    "centroid",
    "area",
    "perimeter",
    "y0",
    "y1"
]
particle_df = pd.DataFrame(columns=columns)


In [23]:
# ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~  #
# loop though length of images that are somewhat usable
for i in range(len(selected_values)-2):
    # pull out selected area + do analysis
    one_crystal = f2ds['ImageData'][:,int(selected_values[i]):int(selected_values[i+1])] # extract 1 crystal
    binary_image = (one_crystal == 0) ## important, convert regions where 0 = True (our bits of interest), all else false
    
    labeled_image, num_features = label(binary_image) # identify connected true areas
    # labeled_image = array, with each true area given a number to identify them
    # num_features = number of unique connected components in image. Have to literally have adjacent pixel, not diagonal (this will make them seperate)
    
    props = regionprops(labeled_image) # creates quick list of properties describing each feature detected in the image.

    if props:
        for particle in props:
            # quickly get rid of tiny particles
            if particle.area >= minimum_area:
                ## basic info
                coords = particle.coords # basically gives coords of each point of interest
                x_values = np.unique(coords[:, 1])
                s_idx = int(selected_values[i] + x_values[0])
                e_idx = int(selected_values[i] + x_values[-1])
                
                ## more complex stats
                spec_region = stats_description(particle)
                ### !!!!!! important part - this is where useful crystals are getting stats + being recorded in the dataframe    
                
                if spec_region and spec_region.major_axis_length * pixel_resolution >= length_threshold:
                    # nice way of saving data - lenth + measurements are correct in microns
                    one_particle_data = {
                            #"image_index": image_index,
                            "name": f'{s_idx}_{particle.label}',
                            "particle_label": particle.label,
                            "start_index": s_idx,
                            "end_index": e_idx,
                            "start_time": str(time_xr['utc_time'][s_idx].values).split('T')[1], # more friendly time
                            "end_time": str(time_xr['utc_time'][e_idx].values).split('T')[1],
                        
                            #"start_time": time_xr['utc_time'][s_idx],  # assuming 'time_xr' is pre-defined and syncs with indices
                            #"end_time": time_xr['utc_time'][e_idx],
                            "major_axis_length": spec_region.major_axis_length * pixel_resolution,
                            "minor_axis_length": spec_region.minor_axis_length * pixel_resolution,
                            "orientation": spec_region.orientation,
                            "centroid": spec_region.centroid,
                            "area": (spec_region.area * (pixel_resolution**2)),
                            "perimeter": (spec_region.perimeter * pixel_resolution),
                            "y0": coords[0][0],
                            "y1": coords[-1][0]
                            }
                    one_particle_data_df = pd.DataFrame([one_particle_data])
                    particle_df = pd.concat([particle_df, one_particle_data_df], ignore_index=True)

/tmp/ipykernel_544/1662605905.py:51: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  particle_df = pd.concat([particle_df, one_particle_data_df], ignore_index=True)


IndexError: index 316672 is out of bounds for axis 0 with size 300000

In [24]:
particle_df

,name,particle_label,start_index,end_index,start_time,end_time,major_axis_length,minor_axis_length,orientation,centroid,area,perimeter,y0,y1
0,15_82,82,15,21,16:51:05.387000000,16:51:05.387000000,345.340872,53.001538,-0.036097,"(18.294117647058822, 3.764705882352941)",8500.0,696.923882,97,127
1,1378_22,22,1378,1413,16:51:07.509000000,16:51:07.555000000,400.146102,332.045327,-0.155526,"(20.21533923303835, 18.13765978367748)",101700.0,1593.675324,69,112
2,10304_41,41,10304,10353,16:55:43.208000000,16:55:43.286000000,491.870635,442.699181,0.068674,"(25.55128205128205, 21.843101343101342)",163800.0,2298.589104,18,69
3,17025_1,1,17025,17071,16:55:53.098000000,16:55:53.207000000,480.666215,422.675017,-0.052924,"(26.03423680456491, 22.95506419400856)",140200.0,1999.177849,83,127
4,17220_1,1,17220,17242,16:55:53.488000000,16:55:53.535000000,312.156569,85.096358,-0.898120,"(9.671717171717171, 11.247474747474747)",19800.0,719.766594,105,124
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
440,226939_1,1,226939,226980,19:11:42.924000000,19:11:43.002000000,535.587700,394.298759,0.057206,"(27.95673671199011, 20.38442521631644)",161800.0,2067.878426,0,54
441,233967_1,1,233967,234001,19:11:53.126000000,19:11:53.173000000,341.091497,213.956982,-1.537083,"(8.150735294117647, 16.625)",54400.0,957.340187,0,20
442,236041_1,1,236041,236066,19:11:55.544000000,19:11:55.560000000,316.975839,250.642476,-0.063272,"(15.415730337078651, 12.321027287319422)",62300.0,940.832611,5,36
443,236163_1,1,236163,236254,19:11:55.654000000,19:11:55.747000000,955.150238,351.697393,1.498493,"(14.213487794786927, 48.450144807612745)",241700.0,3000.010460,0,37


In [ ]:
## image saving:
for index, row in particle_df.iterrows():
    # extract image data from main h5 file
    extract_crystal_image = f2ds['ImageData'][row['y0']-2:row['y1']+3,row['start_index']-2:row['end_index']+3] # a couple of points have been added around the image, as it appears to cut it off
    padded_crystal = add_padding(extract_crystal_image, 120) # add padding 
    save_name = row['name']

    # save
    plt.imsave(f'{save_loc}{save_name}.png', padded_crystal, cmap='Greys')

In [25]:
# save df of stats
particle_df.to_csv(f'{save_loc}particle_stats.csv', index=False)